# Librerías

In [46]:
import pandas as pd
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

# Lectura de datos

In [47]:
train_file = pd.read_csv('TRAIN.csv')
train = pd.DataFrame(train_file)

Antes de hacer nada, eliminamos los id, que no aportan ninguna información, y las fechas de modificación de las filas.

In [48]:
train = train.drop(columns=['customer_id', 'EOP_DAY'])

# Preprocesado

## Borrado de columnas con un solo valor

Vamos a eliminar las columnas que no tienen más valores que 1, es decir, solo aportan coste de computación y son prescindibles.

In [49]:
class EliminarColumnasConstantes(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        self.columnas_a_mantener = [col for col in X.columns if X[col].nunique() > 1]
        return self

    def transform(self, X):
        return X[self.columnas_a_mantener]

## Tratamiento de valores nulos

Al analizar el dataset, se puede ver a simple vista que hay muchos valores 99999 y -9999 que parecen estar metidos a mano o para no poner un NaN. Como de momento no les podemos sacar ningún significado más que un número sustituto, los transformaremos en NaN para su posterior tratamiento.

In [50]:
class ReemplazarValoresInvalidos(BaseEstimator, TransformerMixin):
    def __init__(self, valores_a_reemplazar=None):
        self.valores_a_reemplazar = valores_a_reemplazar or [99999, -9999]

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X.replace(self.valores_a_reemplazar, np.nan)

Una vez tenemos todos nuestros valores nulos, tenemos que mirar un factor muy importante para elegir nuestra estrategia. En primer lugar, tenemos que ver qué porcentaje de los datos es nulo, es decir, si tenemos una columna con 80% de datos nulos, no tiene rescate ninguno.

Para ello primero eliminamos columnas con alto porcentaje de datos nulos.

In [51]:
class EliminarColumnasConMuchosNaN(BaseEstimator, TransformerMixin):
    def __init__(self, umbral=0.3):
        self.umbral = umbral

    def fit(self, X, y=None):
        self.columnas_a_mantener = X.columns[X.isnull().mean() <= self.umbral]
        return self

    def transform(self, X):
        return X[self.columnas_a_mantener]

Ahora tenemos las columnas que se pueden "restaurar" o más bien falsear para no tener que borrar datos y que no contamine de forma significativa el modelo. Para ello, seguimos estos 2 enfoques:


*   **Caso categórico**: Reemplazamos el valor por la categoría más repetida, es decir, la **moda**.
*   **Caso numérico**: Reemplazamos el valor por la **mediana**.



In [52]:
# --- Preprocesamiento numérico ---
numerical_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median'))
])

# --- Preprocesamiento categórico ---
categorical_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

## Filtrado de datos relevantes

Por último, vamos a quitar las columnas que estén intrínsecamente relacionadas con otras columnas, es decir, que no estén aportando nueva información.

In [53]:
# Paso 5: Eliminar columnas altamente correlacionadas
class EliminarColumnasCorrelacionadas(BaseEstimator, TransformerMixin):
    def __init__(self, umbral=0.9):
        self.umbral = umbral

    def fit(self, X, y=None):
        corr_matrix = X.corr().abs()
        upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
        self.columnas_a_eliminar = [column for column in upper.columns if any(upper[column] > self.umbral)]
        return self

    def transform(self, X):
        return X.drop(columns=self.columnas_a_eliminar)

# Pipeline de preprocesado

Vamos a crear un pipeline que nos permita hacer todo el preprocesado de golpe sin tener que ir paso por paso.

In [54]:
class PipelinePreprocesamiento(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.step1 = EliminarColumnasConstantes()
        self.step2 = ReemplazarValoresInvalidos()
        self.step3 = EliminarColumnasConMuchosNaN(umbral=0.3)
        self.step5 = EliminarColumnasCorrelacionadas(umbral=0.9)
        self.preprocesador_columnas = None
        self.column_names_ = None  # Para guardar los nombres finales

    def fit(self, X, y=None):
        # Paso 1 a 3
        X_clean = self.step1.fit_transform(X)
        X_clean = self.step2.fit_transform(X_clean)
        X_clean = self.step3.fit_transform(X_clean)

        # Detección de tipos
        self.columnas_num = X_clean.select_dtypes(include='number').columns.tolist()
        self.columnas_cat = X_clean.select_dtypes(exclude='number').columns.tolist()

        # ColumnTransformer
        self.preprocesador_columnas = ColumnTransformer([
            ('num', numerical_pipeline, self.columnas_num),
            ('cat', categorical_pipeline, self.columnas_cat)
        ])

        # Ajustar column transformer
        self.preprocesador_columnas.fit(X_clean)

        # Obtener nombres de columnas finales
        num_features = self.columnas_num
        cat_features = self.preprocesador_columnas.named_transformers_['cat']\
                          .named_steps['encoder']\
                          .get_feature_names_out(self.columnas_cat).tolist()

        self.column_names_ = num_features + cat_features

        # Para correlación (paso 5)
        X_encoded = self.preprocesador_columnas.transform(X_clean)
        X_encoded_df = pd.DataFrame(X_encoded, columns=self.column_names_)
        self.step5.fit(X_encoded_df)

        return self

    def transform(self, X):
        X_clean = self.step1.transform(X)
        X_clean = self.step2.transform(X_clean)
        X_clean = self.step3.transform(X_clean)

        X_encoded = self.preprocesador_columnas.transform(X_clean)
        X_encoded_df = pd.DataFrame(X_encoded, columns=self.column_names_)

        X_final = self.step5.transform(X_encoded_df)

        # Retorna DataFrame con nombres limpios
        return pd.DataFrame(X_final, columns=X_encoded_df.columns.drop(self.step5.columnas_a_eliminar))

## Preprocesado del dataset

In [55]:
preprocesador = PipelinePreprocesamiento()
X_limpio = preprocesador.fit_transform(train)

# Exportación de datos

Una vez tenemos los datos limpios, podemos pasar a la exportación del dataset limpio.

In [57]:
X_limpio.to_csv('dataset_jazztel_limpio.csv', index=False)